In [1]:

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, precision_score

from fairlearn.metrics import MetricFrame, selection_rate, true_positive_rate


# Step 2: Load & encode dataset

df = pd.read_csv("biased_gender_loans.csv")

for col in df.select_dtypes(include=['object']).columns:
    df[col] = LabelEncoder().fit_transform(df[col])


# Step 3: Split into features and target

X = df.drop(columns=['bank_loan'])
y = df['bank_loan']
protected_attr = 'sex'     # gender as sensitive feature

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)


# Step 4: Train Logistic Regression model

model = LogisticRegression(max_iter=1000)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

print("Accuracy:", accuracy_score(y_test, y_pred))



# Step 5: Compute fairness metrics using fairlearn

sensitive = X_test[protected_attr]

metrics = {
    "TPR": true_positive_rate,
    "Selection Rate": selection_rate,
    "Precision": precision_score
}

mf = MetricFrame(
    metrics=metrics,
    y_true=y_test,
    y_pred=y_pred,
    sensitive_features=sensitive
)

print("\nMetrics by group:\n", mf.by_group)




# Step 6: Extract Equal Opportunity, Disparate Impact, Predictive Parity

equal_opp = mf.difference(method="between_groups")["TPR"]
dir_ratio = mf.ratio(method="between_groups")["Selection Rate"]
pred_parity = mf.difference(method="between_groups")["Precision"]

print("\nFairness Metrics:")
print("Equal Opportunity Difference:", equal_opp)
print("Disparate Impact Ratio:", dir_ratio)
print("Predictive Parity Difference:", pred_parity)

if dir_ratio < 0.8:
    print("⚠️ DIR < 0.8 → Model may violate fairness rule.")
else:
    print("✔ Model satisfies 80% fairness rule.")



Accuracy: 0.9965

Metrics by group:
           TPR  Selection Rate  Precision
sex                                     
0    0.990196        0.406439   1.000000
1    1.000000        0.197813   0.984925

Fairness Metrics:
Equal Opportunity Difference: 0.009803921568627416
Disparate Impact Ratio: 0.48669862016022675
Predictive Parity Difference: 0.015075376884422065
⚠️ DIR < 0.8 → Model may violate fairness rule.
